In [1]:
# ============================================================================
# SUPERMARKET BILLING SYSTEM
# ============================================================================
# This system handles supermarket billing with multiple discount types
# Features: Item management, multiple discounts, itemized receipts
# ============================================================================

# Cell 1: Import Required Libraries and Initialize Constants
# ============================================================================

from datetime import datetime
import os

# Store configuration constants
STORE_NAME = "SUPER MART"
STORE_ADDRESS = "123 Main Street, City, State"
STORE_PHONE = "Tel: (555) 123-4567"
TAX_RATE = 0.08  # 8% sales tax

# Discount thresholds and rates
DISCOUNT_TIER_1 = 100  # First discount threshold
DISCOUNT_TIER_2 = 200  # Second discount threshold
DISCOUNT_RATE_1 = 0.05  # 5% discount
DISCOUNT_RATE_2 = 0.10  # 10% discount

# Category-based discounts
CATEGORY_DISCOUNTS = {
    'electronics': 0.20,  # 20% off
    'clothing': 0.15,     # 15% off
    'groceries': 0.05,    # 5% off
    'home': 0.10          # 10% off
}

# Quantity discount threshold
BULK_QUANTITY = 5  # Buy 5 or more items
BULK_DISCOUNT = 0.10  # Get 10% off

print("=" * 60)
print("SUPERMARKET BILLING SYSTEM")
print("=" * 60)
print("System initialized successfully!")
print(f"Store: {STORE_NAME}")
print("=" * 60)

SUPERMARKET BILLING SYSTEM
System initialized successfully!
Store: SUPER MART


In [2]:
# Cell 2: Function to Display Available Product Categories
# ============================================================================

def display_categories():
    """
    Display all available product categories with their discount rates.
    This helps users understand what discounts are available.
    """
    print("\n" + "=" * 60)
    print("AVAILABLE PRODUCT CATEGORIES")
    print("=" * 60)
    
    for idx, (category, discount) in enumerate(CATEGORY_DISCOUNTS.items(), 1):
        print(f"{idx}. {category.upper():<15} - {discount*100:.0f}% discount")
    
    print("=" * 60)

In [3]:
# Cell 3: Function to Add Item to Cart
# ============================================================================

def add_item_to_cart(cart):
    """
    Add a new item to the shopping cart.
    
    Parameters:
    cart (list): The shopping cart containing all items
    
    Returns:
    bool: True if item added successfully, False otherwise
    """
    try:
        print("\n" + "-" * 60)
        print("ADD NEW ITEM")
        print("-" * 60)
        
        # Get item name
        item_name = input("Enter item name: ").strip()
        if not item_name:
            print("⚠️  Item name cannot be empty!")
            return False
        
        # Get item quantity
        quantity = int(input("Enter quantity: "))
        if quantity <= 0:
            print("⚠️  Quantity must be greater than 0!")
            return False
        
        # Get item price
        price = float(input("Enter price per unit ($): "))
        if price <= 0:
            print("⚠️  Price must be greater than 0!")
            return False
        
        # Display and select category
        display_categories()
        category = input("Enter item category (or press Enter for 'other'): ").strip().lower()
        
        # If category not in predefined list, set as 'other' with no discount
        if category not in CATEGORY_DISCOUNTS:
            category = 'other'
        
        # Calculate line total
        line_total = quantity * price
        
        # Create item dictionary
        item = {
            'name': item_name,
            'quantity': quantity,
            'price': price,
            'category': category,
            'line_total': line_total
        }
        
        # Add item to cart
        cart.append(item)
        
        print(f"✓ {item_name} added to cart successfully!")
        print(f"  Quantity: {quantity}, Price: ${price:.2f}, Total: ${line_total:.2f}")
        
        return True
        
    except ValueError:
        print("⚠️  Invalid input! Please enter correct values.")
        return False
    except Exception as e:
        print(f"⚠️  Error adding item: {e}")
        return False

In [4]:
# Cell 4: Function to Display Current Cart
# ============================================================================

def display_cart(cart):
    """
    Display all items currently in the shopping cart.
    
    Parameters:
    cart (list): The shopping cart containing all items
    """
    if not cart:
        print("\n🛒 Your cart is empty!")
        return
    
    print("\n" + "=" * 60)
    print("CURRENT SHOPPING CART")
    print("=" * 60)
    print(f"{'Item':<20} {'Qty':<8} {'Price':<12} {'Total':<12}")
    print("-" * 60)
    
    for item in cart:
        print(f"{item['name']:<20} {item['quantity']:<8} "
              f"${item['price']:<11.2f} ${item['line_total']:<11.2f}")
    
    print("=" * 60)
    print(f"Total Items in Cart: {len(cart)}")
    print("=" * 60)


In [5]:
# Cell 5: Function to Calculate Subtotal
# ============================================================================

def calculate_subtotal(cart):
    """
    Calculate the subtotal of all items in the cart before any discounts.
    
    Parameters:
    cart (list): The shopping cart containing all items
    
    Returns:
    float: Subtotal amount
    """
    subtotal = sum(item['line_total'] for item in cart)
    return subtotal


In [6]:
# Cell 6: Function to Apply Total Amount Discount
# ============================================================================

def apply_total_amount_discount(subtotal):
    """
    Apply discount based on the total purchase amount.
    Tier 1: $100+ gets 5% off
    Tier 2: $200+ gets 10% off
    
    Parameters:
    subtotal (float): The subtotal amount before discount
    
    Returns:
    tuple: (discount_amount, discount_percentage, tier_message)
    """
    if subtotal >= DISCOUNT_TIER_2:
        discount_rate = DISCOUNT_RATE_2
        discount_amount = subtotal * discount_rate
        tier_message = f"Total Amount Discount (${DISCOUNT_TIER_2}+)"
        return discount_amount, discount_rate * 100, tier_message
    
    elif subtotal >= DISCOUNT_TIER_1:
        discount_rate = DISCOUNT_RATE_1
        discount_amount = subtotal * discount_rate
        tier_message = f"Total Amount Discount (${DISCOUNT_TIER_1}+)"
        return discount_amount, discount_rate * 100, tier_message
    
    else:
        return 0, 0, None

In [7]:
# Cell 7: Function to Apply Category Discounts
# ============================================================================

def apply_category_discounts(cart):
    """
    Apply discounts based on item categories.
    Each category has a predefined discount rate.
    
    Parameters:
    cart (list): The shopping cart containing all items
    
    Returns:
    tuple: (total_discount_amount, discount_details_list)
    """
    category_discount_total = 0
    discount_details = []
    
    for item in cart:
        category = item['category']
        
        # Check if category has a discount
        if category in CATEGORY_DISCOUNTS:
            discount_rate = CATEGORY_DISCOUNTS[category]
            item_discount = item['line_total'] * discount_rate
            category_discount_total += item_discount
            
            # Store discount details for receipt
            discount_details.append({
                'item': item['name'],
                'category': category,
                'discount_rate': discount_rate * 100,
                'discount_amount': item_discount
            })
    
    return category_discount_total, discount_details

In [8]:
# Cell 8: Function to Apply Bulk Quantity Discount
# ============================================================================

def apply_bulk_discount(cart):
    """
    Apply discount for items purchased in bulk (5 or more of same item).
    
    Parameters:
    cart (list): The shopping cart containing all items
    
    Returns:
    tuple: (total_discount_amount, discount_details_list)
    """
    bulk_discount_total = 0
    discount_details = []
    
    for item in cart:
        # Check if quantity meets bulk threshold
        if item['quantity'] >= BULK_QUANTITY:
            item_discount = item['line_total'] * BULK_DISCOUNT
            bulk_discount_total += item_discount
            
            # Store discount details
            discount_details.append({
                'item': item['name'],
                'quantity': item['quantity'],
                'discount_amount': item_discount
            })
    
    return bulk_discount_total, discount_details


In [9]:
# Cell 9: Function to Calculate All Discounts
# ============================================================================

def calculate_discounts(cart, subtotal):
    """
    Calculate all applicable discounts and return discount breakdown.
    
    Parameters:
    cart (list): The shopping cart containing all items
    subtotal (float): The subtotal amount before discounts
    
    Returns:
    dict: Dictionary containing all discount information
    """
    discount_info = {
        'total_amount_discount': 0,
        'total_amount_rate': 0,
        'total_amount_message': None,
        'category_discount': 0,
        'category_details': [],
        'bulk_discount': 0,
        'bulk_details': [],
        'total_discount': 0
    }
    
    # Apply total amount discount
    amount_disc, amount_rate, amount_msg = apply_total_amount_discount(subtotal)
    discount_info['total_amount_discount'] = amount_disc
    discount_info['total_amount_rate'] = amount_rate
    discount_info['total_amount_message'] = amount_msg
    
    # Apply category discounts
    cat_disc, cat_details = apply_category_discounts(cart)
    discount_info['category_discount'] = cat_disc
    discount_info['category_details'] = cat_details
    
    # Apply bulk quantity discounts
    bulk_disc, bulk_details = apply_bulk_discount(cart)
    discount_info['bulk_discount'] = bulk_disc
    discount_info['bulk_details'] = bulk_details
    
    # Calculate total discount (sum of all discounts)
    discount_info['total_discount'] = (
        discount_info['total_amount_discount'] + 
        discount_info['category_discount'] + 
        discount_info['bulk_discount']
    )
    
    return discount_info

In [10]:
# Cell 10: Function to Calculate Tax
# ============================================================================

def calculate_tax(amount_after_discount):
    """
    Calculate sales tax on the amount after discounts.
    
    Parameters:
    amount_after_discount (float): Amount after all discounts applied
    
    Returns:
    float: Tax amount
    """
    tax = amount_after_discount * TAX_RATE
    return tax

In [11]:
# Cell 11: Function to Generate and Print Receipt
# ============================================================================

def print_receipt(cart, discount_info):
    """
    Generate and display a detailed itemized receipt.
    
    Parameters:
    cart (list): The shopping cart containing all items
    discount_info (dict): Dictionary containing all discount information
    """
    # Clear screen for clean receipt display (optional)
    # os.system('clear' if os.name == 'posix' else 'cls')
    
    # Calculate amounts
    subtotal = calculate_subtotal(cart)
    amount_after_discount = subtotal - discount_info['total_discount']
    tax = calculate_tax(amount_after_discount)
    final_total = amount_after_discount + tax
    
    # Get current date and time
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Print receipt header
    print("\n" + "=" * 70)
    print(f"{STORE_NAME:^70}")
    print(f"{STORE_ADDRESS:^70}")
    print(f"{STORE_PHONE:^70}")
    print("=" * 70)
    print(f"Date: {current_time}")
    print("=" * 70)
    
    # Print itemized list
    print("\nITEMIZED BILL")
    print("-" * 70)
    print(f"{'Item':<25} {'Qty':<8} {'Price':<12} {'Total':<12}")
    print("-" * 70)
    
    for item in cart:
        print(f"{item['name']:<25} {item['quantity']:<8} "
              f"${item['price']:<11.2f} ${item['line_total']:<11.2f}")
        # Show category if it has discount
        if item['category'] in CATEGORY_DISCOUNTS:
            print(f"  └─ Category: {item['category']} "
                  f"({CATEGORY_DISCOUNTS[item['category']]*100:.0f}% discount)")
    
    print("-" * 70)
    
    # Print subtotal
    print(f"{'SUBTOTAL:':<58} ${subtotal:>10.2f}")
    print()
    
    # Print discount breakdown
    if discount_info['total_discount'] > 0:
        print("DISCOUNTS APPLIED:")
        print("-" * 70)
        
        # Total amount discount
        if discount_info['total_amount_discount'] > 0:
            print(f"  {discount_info['total_amount_message']} "
                  f"({discount_info['total_amount_rate']:.0f}%)")
            print(f"{'':>48} -${discount_info['total_amount_discount']:>10.2f}")
        
        # Category discounts
        if discount_info['category_discount'] > 0:
            print(f"\n  Category Discounts:")
            for detail in discount_info['category_details']:
                print(f"    • {detail['item']} ({detail['category']}) "
                      f"- {detail['discount_rate']:.0f}% off")
                print(f"{'':>48} -${detail['discount_amount']:>10.2f}")
        
        # Bulk discounts
        if discount_info['bulk_discount'] > 0:
            print(f"\n  Bulk Purchase Discounts ({BULK_DISCOUNT*100:.0f}% off for {BULK_QUANTITY}+ items):")
            for detail in discount_info['bulk_details']:
                print(f"    • {detail['item']} (Qty: {detail['quantity']})")
                print(f"{'':>48} -${detail['discount_amount']:>10.2f}")
        
        print("-" * 70)
        print(f"{'TOTAL DISCOUNT:':<58} -${discount_info['total_discount']:>10.2f}")
        print()
    
    # Print amount after discount
    print(f"{'AMOUNT AFTER DISCOUNT:':<58} ${amount_after_discount:>10.2f}")
    
    # Print tax
    print(f"{'TAX (' + str(int(TAX_RATE*100)) + '%):':58} ${tax:>10.2f}")
    print("=" * 70)
    
    # Print final total
    print(f"{'TOTAL AMOUNT DUE:':<58} ${final_total:>10.2f}")
    print("=" * 70)
    
    # Savings summary
    if discount_info['total_discount'] > 0:
        savings_percent = (discount_info['total_discount'] / subtotal) * 100
        print(f"\n🎉 YOU SAVED: ${discount_info['total_discount']:.2f} ({savings_percent:.1f}%)")
    
    print("\n" + "=" * 70)
    print("THANK YOU FOR SHOPPING WITH US!".center(70))
    print("PLEASE VISIT AGAIN!".center(70))
    print("=" * 70 + "\n")

In [12]:
# Cell 12: Main Billing System Function
# ============================================================================

def supermarket_billing_system():
    """
    Main function to run the supermarket billing system.
    Handles the entire billing process from item entry to receipt generation.
    """
    # Initialize empty shopping cart
    cart = []
    
    print("\n" + "=" * 60)
    print("WELCOME TO THE SUPERMARKET BILLING SYSTEM")
    print("=" * 60)
    
    # Main loop for adding items
    while True:
        print("\n" + "=" * 60)
        print("MENU OPTIONS")
        print("=" * 60)
        print("1. Add item to cart")
        print("2. View current cart")
        print("3. Complete purchase and generate receipt")
        print("4. Cancel and exit")
        print("=" * 60)
        
        choice = input("\nEnter your choice (1-4): ").strip()
        
        if choice == '1':
            # Add item to cart
            add_item_to_cart(cart)
            
        elif choice == '2':
            # Display current cart
            display_cart(cart)
            
        elif choice == '3':
            # Complete purchase
            if not cart:
                print("\n⚠️  Your cart is empty! Please add items first.")
                continue
            
            # Display final cart
            display_cart(cart)
            
            # Confirm purchase
            confirm = input("\nProceed with checkout? (yes/no): ").strip().lower()
            
            if confirm in ['yes', 'y']:
                # Calculate subtotal
                subtotal = calculate_subtotal(cart)
                
                # Calculate all discounts
                discount_info = calculate_discounts(cart, subtotal)
                
                # Generate and print receipt
                print_receipt(cart, discount_info)
                
                # Ask if user wants to start a new transaction
                new_transaction = input("\nStart a new transaction? (yes/no): ").strip().lower()
                if new_transaction in ['yes', 'y']:
                    cart = []  # Reset cart
                    print("\n✓ New transaction started!")
                else:
                    print("\n✓ Thank you! Exiting system...")
                    break
            else:
                print("✓ Checkout cancelled.")
                
        elif choice == '4':
            # Exit system
            print("\n✓ Thank you! Exiting system...")
            break
            
        else:
            print("\n⚠️  Invalid choice! Please enter 1-4.")

In [ ]:
# Cell 13: Run the Billing System
# ============================================================================

# Call the main function to start the billing system
supermarket_billing_system()


WELCOME TO THE SUPERMARKET BILLING SYSTEM

MENU OPTIONS
1. Add item to cart
2. View current cart
3. Complete purchase and generate receipt
4. Cancel and exit
